In [232]:
import wandb

In [233]:
api = wandb.Api()
entity, project = "tychovdo", "sparse"  
runs = api.runs(entity + "/" + project) 

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains output keys/values for 
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
         if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)
    
data = list(zip(summary_list, config_list, name_list))

In [234]:
def get_runs(model, kron):
    runs = []
    for summary, config, name in data:
        is_kron = config['n_epochs_burnin'] < 100
        if (config['model'] == model) and \
            (is_kron == kron):
            runs.append((dict(summary), dict(config), list(name)))
    return runs

runs = get_runs('f_rpp', True)
print('runs:', len(runs))
run = runs[0]

runs: 3


In [235]:
def print_vars(model):
    run = get_runs(model, True)[0]
    
    summary, config, name = run
    
    largest = []
    for layer in range(8):
        if model == 'd_rpp':
            linear_name = f'hyperparams/prior_prec_conv_net.{layer}.linear.weight'
            conv_name = f'hyperparams/prior_prec_conv_net.{layer}.conv.weight'
            linear = summary[linear_name]
            conv = summary[conv_name]
            print(f"{layer}: {linear:10.2f}\t{conv:8.8f}")
            
            if linear < conv:
                largest.append('FC-')
            else:
                largest.append('CONV-')
        elif model == 'f_rpp':
            linearu_name = f'hyperparams/prior_prec_conv_net.{layer}.linear.lin_U.weight'
            linearu = summary[linearu_name]
            
            linearv_name = f'hyperparams/prior_prec_conv_net.{layer}.linear.lin_V.weight'
            linearv = summary[linearv_name]
            
            conv_name = f'hyperparams/prior_prec_conv_net.{layer}.conv.weight'
            conv = summary[conv_name]
        
            print(f"{layer}: {linearu:8.4f}\t{linearv:8.8f}\t{conv:8.8f}")
        
            if (conv < linearu) or (conv < linearv):
                largest.append('CONV-')
            else:
                largest.append('FC-')
        
    largest.append('FINAL')
    print(''.join(largest))
    
print('FC          \t+ CONV')
print_vars('d_rpp')

print('\n')
print('F-FC (U) \t+ F-FC (V)\t + CONV')
print_vars('f_rpp')

FC          	+ CONV
0: 6103090.00	0.07465822
1: 2653141.75	1.16207194
2: 7176070.50	16.08159065
3: 22742260.00	244.40238953
4: 30352452.00	5442.07324219
5: 33993908.00	3492.19628906
6:     578.21	27184420.00000000
7: 35857912.00	1.65367436
CONV-CONV-CONV-CONV-CONV-CONV-FC-CONV-FINAL


F-FC (U) 	+ F-FC (V)	 + CONV
0: 40080.7305	0.00183259	0.14375170
1: 43148.7266	0.00716106	3.25895357
2: 129810.3047	0.00685028	53.23348236
3: 800174.0625	0.16992638	6965.95605469
4: 482410.0625	0.46733886	97.60765839
5:   1.1085	1.26341522	16.68072128
6:   0.0636	0.81319302	784.90270996
7:   0.0001	19624.19726562	3.56260467
CONV-CONV-CONV-CONV-CONV-FC-FC-CONV-FINAL


In [236]:
conv_net.7.linear.lin_V.weight

SyntaxError: invalid syntax (2710877218.py, line 1)

In [255]:
def print_vars(model):
    run = get_runs(model, True)[0]
    
    summary, config, name = run
    
    largest = []
    for layer in range(8):
        if model == 'd_rpp':
            linear_name = f'params/conv_net.{layer}.linear.weight'
            conv_name = f'params/conv_net.{layer}.conv.weight'
            linear = summary[linear_name]
            conv = summary[conv_name]
            print(f"{layer}: {linear:10.18f}   \t{conv:8.8f}")

        elif model == 'f_rpp':
            
            linearu_name = f'params/conv_net.{layer}.linear.lin_U.weight'
            linearu = summary[linearu_name]
            
            linearv_name = f'params/conv_net.{layer}.linear.lin_V.weight'
            linearv = summary[linearv_name]
            
            conv_name = f'params/conv_net.{layer}.conv.weight'
            conv = summary[conv_name]
        
            print(f"{layer}: {linearu:8.15f}\t{linearv:8.12f}\t{conv:8.8f}")

    
print('FC                          \t+ CONV')
print_vars('d_rpp')

print('\n')
print('F-FC (U)            \t+ F-FC (V)    \t  + CONV')
print_vars('f_rpp')

FC                          	+ CONV
0: 0.000000000000000540   	11.00202908
1: 0.000000000000013544   	0.60940274
2: 0.000000000000008409   	0.03992084
3: 0.000000000000041448   	0.00205524
4: 0.000000000000257826   	0.00007488
5: 0.000000000000107112   	0.00004042
6: 0.000012328107841313   	0.00000000
7: 0.000000000000001133   	0.00263015


F-FC (U)            	+ F-FC (V)    	  + CONV
0: 0.000000002244836	0.917117691040	6.19126587
1: 0.000000000153714	0.063998484612	0.09114730
2: 0.000000000065362	0.036372072697	0.00562219
3: 0.000000000587946	0.029791200161	0.00002159
4: 0.000000000009757	0.008005650043	0.00046595
5: 0.005006082057953	0.007090061307	0.00015731
6: 0.047147378921509	0.007864879370	0.00000002
7: 0.230331478118896	0.000000000423	0.00104415


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wand